In [1]:
# 경고 메시지가 안나오게..
import warnings
warnings.filterwarnings('ignore')

# 기본
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')
import missingno

# KFold
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 교차 검증 함수
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

# 학습 데이터와 검증데이터로 나누는 함수
from sklearn.model_selection import train_test_split

# 데이터 전처리
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 하이퍼 파라미터 튜닝
from sklearn.model_selection import GridSearchCV

# 평가함수
from sklearn.metrics import accuracy_score

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

# 군집
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift

# 차원축소
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 딥러닝
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

from keras.utils import np_utils
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Conv1D
from keras.layers import MaxPooling1D

# 다중분류를 위한 핫-윈 인코더
from keras.utils import to_categorical

# 저장된 딥러닝 모델을 복구하는 함수
from keras.models import load_model

# epoch마다 모델을 저장하는 함수
from keras.callbacks import ModelCheckpoint

# 더이상 성능 향상이 이루어지지 않는다면 조기 중단시킬 수 있는 함수
from keras.callbacks import EarlyStopping

# 문장을 잘라준다.
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence

# 저장
import pickle

# 시간 모듈
import time

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

# gpu 사용 초기화 및 할당
gpus= tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [2]:
# seed 설정
np.random.seed(3)
tf.random.set_seed(3)

In [3]:
# 데이터를 읽어온다
df1 = pd.read_csv('../../data/naver_star_data.csv')
df1.head()

,평점,평가글,작성자,작성날짜,공감수,비공감수
0,1.0,"B급 이하전편보다 퇴보된 CG, 중구난방식 연출, 러닝타임 늘리기 위한 쓸모없는 컷...",코꾸뇨우옹(pott****),2020.12.24 09:08,873.0,289.0
1,3.0,이딴 영화가 평점 8점 후반대라는 게 믿기지 않는다. 역시 네이버 평점은 믿고 걸러...,juum****,2020.12.23 22:19,620.0,232.0
2,6.0,역시 원더우먼 영화는 주연배우 갤가돗과 크리스파인이 다 살리네.. 감독은 확실히 영...,없음(jymi****),2020.12.23 17:05,480.0,168.0
3,2.0,진짜 개노잼이다.. 1편이랑 같은 감독맞나?러닝타임도 길어서 개지루함 ㄹㅇ,민중의빠따(gkst****),2020.12.23 22:51,433.0,143.0
4,1.0,히어로물의 액션을 기대했음. 그러나 졸렬한 액션과 이상한 전개로 마지막 영화관을 나...,시리우스(sojh****),2020.12.23 13:34,412.0,198.0


In [4]:
# 평점과 평가글만 추출한다
df2 = df1[['평점', '평가글']]
df2.head()

,평점,평가글
0,1.0,"B급 이하전편보다 퇴보된 CG, 중구난방식 연출, 러닝타임 늘리기 위한 쓸모없는 컷..."
1,3.0,이딴 영화가 평점 8점 후반대라는 게 믿기지 않는다. 역시 네이버 평점은 믿고 걸러...
2,6.0,역시 원더우먼 영화는 주연배우 갤가돗과 크리스파인이 다 살리네.. 감독은 확실히 영...
3,2.0,진짜 개노잼이다.. 1편이랑 같은 감독맞나?러닝타임도 길어서 개지루함 ㄹㅇ
4,1.0,히어로물의 액션을 기대했음. 그러나 졸렬한 액션과 이상한 전개로 마지막 영화관을 나...


In [25]:
# 테스트용 코드 - 학습량이 너무 많기 때문에 램덤하게 500개만 추출한다
df2 = df2.sample(n=500).reset_index(drop=True)
df2.head()

,평점,평가글
0,10.0,NaN
1,10.0,"울리고, 웃기고 우왕~ 굳!!"
2,2.0,불륜극혐ㅁㅁㅁㅁ10자
3,1.0,일본 특유의 클리ㅛ셰에 여주는 무슨 상상속인물마냥 지나치게 밝고 쾌활하고 억지웃음에...
4,10.0,진한여운이 남는 기분좋아지는영화네요


In [26]:
# 결측치 확인 
df2.isna().sum()

평점      0
평가글    13
dtype: int64

In [27]:
# 결측치 제거
df2.dropna(inplace=True)
df2.isna().sum()

평점     0
평가글    0
dtype: int64

In [28]:
# 글 인덱스를 평점을 기준으로 나눠서 추출한다
a1 = df2.query('평점<=5').index
a2 = df2.query('평점>5').index

# 새로운 값을 세팅한다(0:부정, 1:긍정)
df2.loc[a1, '평점'] = 0
df2.loc[a2, '평점'] = 1

In [29]:
df2['평점'].value_counts()

1.0    415
0.0     72
Name: 평점, dtype: int64

In [30]:
# 데이터를 분리한다
docs = df2['평가글'].values
classes = df2['평점'].values

In [31]:
# 단어 사전을 생성한다
token = Tokenizer()
token.fit_on_texts(docs)
token.word_index

{'영화': 1,
 '너무': 2,
 '정말': 3,
 '진짜': 4,
 '이': 5,
 '영화를': 6,
 '더': 7,
 '잘': 8,
 '수': 9,
 '스토리': 10,
 '좋은': 11,
 '영화는': 12,
 '그': 13,
 '많이': 14,
 '것': 15,
 '본': 16,
 '왜': 17,
 '그냥': 18,
 '한': 19,
 '마지막': 20,
 '좋았어요': 21,
 '배우들': 22,
 '최고': 23,
 '좀': 24,
 '다': 25,
 '감동': 26,
 '오랜만에': 27,
 '그리고': 28,
 '다시': 29,
 '이런': 30,
 '스토리가': 31,
 '봤는데': 32,
 '진심': 33,
 '내내': 34,
 '솔직히': 35,
 '모두': 36,
 '내가': 37,
 '모든': 38,
 '역시': 39,
 '영화가': 40,
 '꼭': 41,
 '없는': 42,
 '대한': 43,
 '스토리도': 44,
 '최고의': 45,
 '보고': 46,
 '봤어요': 47,
 '뭔가': 48,
 '같은': 49,
 '배우들의': 50,
 '연기': 51,
 '아닌': 52,
 '연기도': 53,
 '처음': 54,
 '뻔한': 55,
 '재밌게': 56,
 '영화네요': 57,
 '되는': 58,
 '꿈을': 59,
 '감독의': 60,
 '근데': 61,
 '가슴이': 62,
 '그렇게': 63,
 '내': 64,
 '가장': 65,
 '아': 66,
 '눈물': 67,
 '아주': 68,
 '있는': 69,
 '저는': 70,
 '생각이': 71,
 '보는': 72,
 '이렇게': 73,
 '내용이': 74,
 '남는': 75,
 '없다': 76,
 '나': 77,
 '재밌어요': 78,
 '봤습니다': 79,
 '이건': 80,
 '모르겠다': 81,
 '같아요': 82,
 '음악이': 83,
 '그저': 84,
 '끝까지': 85,
 'ㅠㅠ': 86,
 '노래': 87,
 '다른': 88,
 '간만에': 89,
 '억지

In [32]:
# 생성한 단어사전을 기준으로 단어들을 숫자로 변환한다
X = token.texts_to_sequences(docs)
X[0]

[566, 567, 568, 262]

In [33]:
# 문장당 최대 단어 개수를 구한다
max_length = 0

for str1 in X:
    # 현재 문장의 단어수를 가져온다
    a1 = len(str1)
    # 현재 문장의 단어수가 이전 최대 단어수 보다 크면
    # 값을 덮어씌운다
    if max_length < a1:
        max_length = a1
    
max_length

35

In [34]:
# 패딩 서로 길이가 다른 리스트(단어수)의 개수를 맞춰준다
# 부족한 부분을 0으로 채워준다
padded_x = pad_sequences(X, max_length)
padded_x

array([[   0,    0,    0, ...,  567,  568,  262],
       [   0,    0,    0, ...,    0,    0,  569],
       [   0,    0,    0, ...,  582,  583,  584],
       ...,
       [   0,    0,    0, ..., 3347, 3348,  565],
       [   0,    0,    0, ...,  236, 3359, 3360],
       [   0,    0,    0, ...,    1,    4, 3361]])

In [35]:
# 단어의 개수에 1을 더한 값을 구한다
word_size = len(token.word_index) + 1
word_size

3362

In [36]:
# 모델 설정
model = Sequential()
model.add(Embedding(word_size, 100))
model.add(Dropout(0.5))
model.add(Conv1D(64, 5, padding='valid', activation='relu', strides=1))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(55))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [37]:
# 모델 정보
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         336200    
_________________________________________________________________
dropout (Dropout)            (None, None, 100)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 64)          32064     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 64)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 55)                26400     
_________________________________________________________________
dense (Dense)                (None, 1)                 56        
_________________________________________________________________
activation (Activation)      (None, 1)                 0

In [38]:
# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
# 데이터를 나눈다
X_train, X_test, y_train, y_test = train_test_split(padded_x, classes, )

In [40]:
# 학습 시작
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=100, epochs=5)

Epoch 1/5
4/4 [==============================] - 5s 354ms/step - loss: 0.6594 - accuracy: 0.8414 - val_loss: 0.5971 - val_accuracy: 0.8361
Epoch 2/5
4/4 [==============================] - 0s 19ms/step - loss: 0.5363 - accuracy: 0.8607 - val_loss: 0.4821 - val_accuracy: 0.8361
Epoch 3/5
4/4 [==============================] - 0s 18ms/step - loss: 0.4282 - accuracy: 0.8540 - val_loss: 0.4154 - val_accuracy: 0.8361
Epoch 4/5
4/4 [==============================] - 0s 19ms/step - loss: 0.3658 - accuracy: 0.8767 - val_loss: 0.4403 - val_accuracy: 0.8361
Epoch 5/5
4/4 [==============================] - 0s 18ms/step - loss: 0.4395 - accuracy: 0.8483 - val_loss: 0.4215 - val_accuracy: 0.8361


In [41]:
# 정확도 측정
model.evaluate(X_test, y_test)[1]

4/4 [==============================] - 0s 20ms/step - loss: 0.4215 - accuracy: 0.8361


0.8360655903816223